In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
DATASET_PATH = '/content/drive/MyDrive/archive'  # adjust if your "archive" folder is in a different path


In [3]:
!pip install librosa pandas scikit-learn tensorflow


In [4]:
import pandas as pd

meta = pd.read_csv(DATASET_PATH + '/UrbanSound8K.csv')
print(meta.head())



      slice_file_name    fsID  start        end  salience  fold  classID  \
0    100032-3-0-0.wav  100032    0.0   0.317551         1     5        3   
1  100263-2-0-117.wav  100263   58.5  62.500000         1     5        2   
2  100263-2-0-121.wav  100263   60.5  64.500000         1     5        2   
3  100263-2-0-126.wav  100263   63.0  67.000000         1     5        2   
4  100263-2-0-137.wav  100263   68.5  72.500000         1     5        2   

              class  
0          dog_bark  
1  children_playing  
2  children_playing  
3  children_playing  
4  children_playing  


In [5]:
import librosa
import numpy as np

def extract_features(file_path, sr=22050, n_mfcc=40):
    audio, _ = librosa.load(file_path, sr=sr)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean


In [ ]:
features = []
labels = []
for idx, row in meta.iterrows():
    fold = row['fold']
    file_name = row['slice_file_name']
    file_path = f"{DATASET_PATH}/fold{fold}/{file_name}"
    label = row['classID']
    try:
        mfccs = extract_features(file_path)
        features.append(mfccs)
        labels.append(label)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

X = np.array(features)
y = np.array(labels)


/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [8]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)


RandomForestClassifier()

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)


RandomForestClassifier()

In [10]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.887807670291929
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       200
           1       0.97      0.81      0.89        86
           2       0.78      0.89      0.83       200
           3       0.83      0.81      0.82       200
           4       0.90      0.89      0.89       200
           5       0.96      0.95      0.96       200
           6       0.98      0.64      0.77        75
           7       0.91      0.94      0.92       200
           8       0.93      0.96      0.94       186
           9       0.79      0.85      0.82       200

    accuracy                           0.89      1747
   macro avg       0.90      0.87      0.88      1747
weighted avg       0.89      0.89      0.89      1747



In [11]:
import joblib
joblib.dump(rf, "/content/drive/MyDrive/random_forest_model.pkl")


['/content/drive/MyDrive/random_forest_model.pkl']

In [12]:
import librosa
import numpy as np

def extract_features(file_path, sr=22050, n_mfcc=40):
    audio, _ = librosa.load(file_path, sr=sr)
    if len(audio) < 2048:
        return None
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean


In [13]:
import joblib

model = joblib.load('/content/drive/MyDrive/random_forest_model.pkl')


In [14]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/your_trained_model.h5')


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/your_trained_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [17]:
import joblib
joblib.dump(rf, '/content/drive/MyDrive/urban8k_rf_model.pkl')


['/content/drive/MyDrive/urban8k_rf_model.pkl']

In [18]:
import joblib
rf = joblib.load('/content/drive/MyDrive/urban8k_rf_model.pkl')


In [19]:
# 1. Feature extraction function (must use the same as in training)
def extract_features(file_path, sr=22050, n_mfcc=40):
    import librosa
    import numpy as np
    audio, _ = librosa.load(file_path, sr=sr)
    if len(audio) < 2048:
        return None
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

# 2. Load Random Forest model
import joblib
model = joblib.load('/content/drive/MyDrive/urban8k_rf_model.pkl')

# 3. Load label names (UrbanSound8K classes)
import pandas as pd
meta = pd.read_csv('/content/drive/MyDrive/archive/UrbanSound8K.csv')
class_map = dict(zip(meta['classID'], meta['class']))

# 4. Predict on a new audio file
test_path = '/content/drive/MyDrive/random_test.wav'  # <-- your file here
features = extract_features(test_path)
if features is not None:
    pred = model.predict([features])[0]
    print("Predicted class label:", class_map[pred])
else:
    print("Audio file too short or not valid for feature extraction.")


Predicted class label: car_horn


In [20]:
# 1. Feature extraction function (must use the same as in training)
def extract_features(file_path, sr=22050, n_mfcc=40):
    import librosa
    import numpy as np
    audio, _ = librosa.load(file_path, sr=sr)
    if len(audio) < 2048:
        return None
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

# 2. Load Random Forest model
import joblib
model = joblib.load('/content/drive/MyDrive/urban8k_rf_model.pkl')

# 3. Load label names (UrbanSound8K classes)
import pandas as pd
meta = pd.read_csv('/content/drive/MyDrive/archive/UrbanSound8K.csv')
class_map = dict(zip(meta['classID'], meta['class']))

# 4. Predict on a new audio file
test_path = '/content/drive/MyDrive/test_audio_2.wav'  # <-- your file here
features = extract_features(test_path)
if features is not None:
    pred = model.predict([features])[0]
    print("Predicted class label:", class_map[pred])
else:
    print("Audio file too short or not valid for feature extraction.")


Predicted class label: drilling


In [22]:
# 1. Feature extraction function (must use the same as in training)
def extract_features(file_path, sr=22050, n_mfcc=40):
    import librosa
    import numpy as np
    audio, _ = librosa.load(file_path, sr=sr)
    if len(audio) < 2048:
        return None
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

# 2. Load Random Forest model
import joblib
model = joblib.load('/content/drive/MyDrive/urban8k_rf_model.pkl')

# 3. Load label names (UrbanSound8K classes)
import pandas as pd
meta = pd.read_csv('/content/drive/MyDrive/archive/UrbanSound8K.csv')
class_map = dict(zip(meta['classID'], meta['class']))

# 4. Predict on a new audio file
test_path = '/content/drive/MyDrive/test_audio_3.wav'  # <-- your file here
features = extract_features(test_path)
if features is not None:
    pred = model.predict([features])[0]
    print("Predicted class label:", class_map[pred])
else:
    print("Audio file too short or not valid for feature extraction.")


Predicted class label: street_music


In [25]:
# 1. Feature extraction function (must use the same as in training)
def extract_features(file_path, sr=22050, n_mfcc=40):
    import librosa
    import numpy as np
    audio, _ = librosa.load(file_path, sr=sr)
    if len(audio) < 2048:
        return None
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

# 2. Load Random Forest model
import joblib
model = joblib.load('/content/drive/MyDrive/urban8k_rf_model.pkl')

# 3. Load label names (UrbanSound8K classes)
import pandas as pd
meta = pd.read_csv('/content/drive/MyDrive/archive/UrbanSound8K.csv')
class_map = dict(zip(meta['classID'], meta['class']))

# 4. Predict on a new audio file
test_path = '/content/drive/MyDrive/test_audio_4.wav'  # <-- your file here
features = extract_features(test_path)
if features is not None:
    pred = model.predict([features])[0]
    print("Predicted class label:", class_map[pred])
else:
    print("Audio file too short or not valid for feature extraction.")


Predicted class label: dog_bark
